In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
import numpy as np
import json
from llama_index import Document
import openai

openai.api_key = os.environ.get("OPENAI_API_KEY")
from llama_index.vector_stores import PineconeVectorStore,MilvusVectorStore
from PyPDF2 import PdfReader

In [4]:
import time
from PyPDF2 import PdfReader
from PIL import Image

import os


# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
import openai
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import ChatGLM
from langchain.prompts import PromptTemplate

In [5]:
# default endpoint_url for a local deployed ChatGLM api server
endpoint_url = "https://5448-35-227-160-165.ngrok-free.app"

# direct access endpoint in a proxied environment
# os.environ['NO_PROXY'] = '127.0.0.1'

llm = ChatGLM(
    endpoint_url=endpoint_url,
    max_token=80000,
    history=[],
    top_p=0.9,
)

In [3]:
uri=os.environ.get("uri")
token=os.environ.get("token")
collection_name=os.environ.get("MILVUS_COLLECTION_NAME")
dim=os.environ.get("VECTOR_DIMENSION")
vector_field=os.environ.get("VECTOR_FIELD")
doc_id_field=os.environ.get("DOC_ID_FIELD")
text_key=os.environ.get("TEXT_KEY")
pincone_env=os.environ.get("PINCONE_ENV")
pincone_key=os.environ.get("PINCONE_KEY")

In [4]:
with open("data.json","r") as f:
    data=json.load(f)

In [5]:
data[0]

{'pdf_name': 'AVATAR_JamesCameron.pdf',
 'pdf_unique_id': 102813233188629339,
 'text': 'AVATAR  written by  James Cameron                                 ALL RIGHTS RESERVED. COPYRIGHT ©2007 TWENTIETH CENTURY FOX FILM CORPORATION.  NO PORTION OF THIS SCRIPT MAY BE PERFORMED, PUBLISHED, REPRODUCED, SOLD, OR DISTRIBUTED BY ANY MEANS, OR QUOTED OR PUBLISHED IN ANY MEDIUM, INCLUDING ANY WEB SITE, WITHOUT THE PRIOR WRITTEN CONSENT OF TWENTIETH CENTURY FOX FILM CORPORATION. \u2029THE SOUND OF DRUMS, from a great distance, growing louder.FADE IN:WE ARE FLYING through mist, a dimly glimpsed forest below.VOICE (V.O.)When I was lying there in the VA hospital, with a big hole blown through the middle of my life, I started having these dreams of flying.We are very low over the forest now, gliding fast, the drums BUILDING to a PEAK --VOICE (V.O.)Sooner or later though, you always have to wake up...CUT TO:EXT. CITY - NIGHTA SCREECH OF BRAKES as a vehicle WIPES FRAME, revealing --JAKE SULLY, a scarre

In [10]:

from llama_index import Document

docs = []

for row in data:
    docs.append(Document(
        text=row['text'],
        doc_id=row['text_id'],
        extra_info={'pdf_name': row['pdf_name'],"pdf_unique_id":row['pdf_unique_id']}
    ))
docs[0]

Document(id_='1', embedding=None, metadata={'pdf_name': 'AVATAR_JamesCameron.pdf', 'pdf_unique_id': 102813233188629339}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='b9887d9a7048cba00f32ba3a2ca94bbaf13e37561eb4420e2db37f366b3e63bd', text='AVATAR  written by  James Cameron                                 ALL RIGHTS RESERVED. COPYRIGHT ©2007 TWENTIETH CENTURY FOX FILM CORPORATION.  NO PORTION OF THIS SCRIPT MAY BE PERFORMED, PUBLISHED, REPRODUCED, SOLD, OR DISTRIBUTED BY ANY MEANS, OR QUOTED OR PUBLISHED IN ANY MEDIUM, INCLUDING ANY WEB SITE, WITHOUT THE PRIOR WRITTEN CONSENT OF TWENTIETH CENTURY FOX FILM CORPORATION. \u2029THE SOUND OF DRUMS, from a great distance, growing louder.FADE IN:WE ARE FLYING through mist, a dimly glimpsed forest below.VOICE (V.O.)When I was lying there in the VA hospital, with a big hole blown through the middle of my life, I started having these dreams of flying.We are very low over the forest now, gliding fast, the 

In [ ]:
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser()

nodes = parser.get_nodes_from_documents(docs)
nodes[0]
     

In [17]:

import pinecone



# initialize connection to pinecone
pinecone.init(
    api_key=os.environ.get('PINCONE_KEY'),
    environment=os.environ.get('PINCONE_ENV')
)

# create the index if it does not exist already
index_name = 'llama-index-intro'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=768,
        metric='cosine'
    )

# connect to the index
pinecone_index = pinecone.Index(index_name)

In [18]:

from llama_index.vector_stores import PineconeVectorStore

# we can select a namespace (acts as a partition in an index)
namespace = '' # default namespace

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

In [ ]:
from llama_index import GPTVectorStoreIndex, StorageContext, ServiceContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# setup our storage (vector db)
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)
# setup the index/query process, ie the embedding model (and completion if used)
embed_model = HuggingFaceEmbeddings()
service_context = ServiceContext.from_defaults(embed_model=embed_model)

index = GPTVectorStoreIndex.from_documents(
    docs[:20], storage_context=storage_context,
    service_context=service_context
)

In [23]:
docs[0].text

'AVATAR  written by  James Cameron                                 ALL RIGHTS RESERVED. COPYRIGHT ©2007 TWENTIETH CENTURY FOX FILM CORPORATION.  NO PORTION OF THIS SCRIPT MAY BE PERFORMED, PUBLISHED, REPRODUCED, SOLD, OR DISTRIBUTED BY ANY MEANS, OR QUOTED OR PUBLISHED IN ANY MEDIUM, INCLUDING ANY WEB SITE, WITHOUT THE PRIOR WRITTEN CONSENT OF TWENTIETH CENTURY FOX FILM CORPORATION. \u2029THE SOUND OF DRUMS, from a great distance, growing louder.FADE IN:WE ARE FLYING through mist, a dimly glimpsed forest below.VOICE (V.O.)When I was lying there in the VA hospital, with a big hole blown through the middle of my life, I started having these dreams of flying.We are very low over the forest now, gliding fast, the drums BUILDING to a PEAK --VOICE (V.O.)Sooner or later though, you always have to wake up...CUT TO:EXT. CITY - NIGHTA SCREECH OF BRAKES as a vehicle WIPES FRAME, revealing --JAKE SULLY, a scarred and scruffy combat vet, sitting in a beat up carbon-fiber wheelchair.  At 22, his eye

In [24]:
query_engine = index.as_query_engine()
res = query_engine.query("who is writtern by avtar?")
print(res)

James Cameron is the author of the script for the movie "Avatar".


In [29]:
from llama_index.node_parser import SimpleNodeParser

In [31]:
parser = SimpleNodeParser()
new_nodes = parser.get_nodes_from_documents(docs[30:len(docs)])

# Add nodes to the existing index
print("Adding new nodes to the existing index...")
index.insert_nodes(new_nodes)

Adding new nodes to the existing index...


Upserted vectors:   0%|          | 0/225 [00:00<?, ?it/s]

In [59]:
query_engine = index.as_query_engine()
res = query_engine.query("how many film characters give me list of names avatar find out from the dialouges")

In [60]:
res.response

'There are several film characters mentioned in the dialogue, but the exact number is not specified. Some of the characters mentioned include Jake, Max, Grace, Norm, Trudy Chacon, and the Colonel.'

In [61]:
import textwrap 
  
  
# Wrap this text. 
wrapper = textwrap.TextWrapper(width=50) 
  
word_list = wrapper.wrap(text=res.response) 
  
# Print each line. 
for element in word_list: 
    print(element) 

There are several film characters mentioned in the
dialogue, but the exact number is not specified.
Some of the characters mentioned include Jake,
Max, Grace, Norm, Trudy Chacon, and the Colonel.
